In [2]:
# default_exp core

# Core

> API details.

In [3]:
#hide
from nbdev.showdoc import *

In [5]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code.ipynb.


In [8]:
#export

import pkgutil
import importlib
import time
import nbdev.imports
from nbdev.export import *

def get_package_name():
    "Returns name of the current package"
    # see logic in https://github.com/fastai/nbdev/blob/master/nbdev/export.py
    return str(nbdev.imports.Config().lib_path).split('/')[-1]

def reload_package_modules(package_name):
    "Dynamically reloads package `package_name`"
    for _, module_name, _ in (pkgutil.iter_modules([package_name])):
        module = importlib.import_module(package_name + '.' + module_name)
        importlib.reload(module)
        
def export_and_reload_all():
    "Converts all notebooks to modules, and reloads all modules"
    if nbdev.imports.in_ipython():
        notebook2script()
        time.sleep(1)
        reload_package_modules(get_package_name())

In [9]:
#hide
get_package_name()

'gcp_runner'

In [10]:
#export
import IPython.core
import json
import os.path
import re
import ipykernel
import requests

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_path():
    """
    Return the full path of the jupyter notebook.
    !!! Doesn't work from VS code.
    """
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        try:
            response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                    params={'token': ss.get('token', '')}, timeout=1)
            for nn in json.loads(response.text):
                if nn['kernel']['id'] == kernel_id:
                    relative_path = nn['notebook']['path']
                    return os.path.join(ss['notebook_dir'], relative_path)
        except Exception as e:
            pass


In [12]:
#hide
get_notebook_path()

'/usr/local/google/home/alekseyv/vlasenkoalexey/gcp_runner/00_core.ipynb'

In [13]:
#export
import nbdev.export

def find_default_export_for_notebook(notebook_path):
    nb = nbdev.export.read_nb(notebook_path)
    default = nbdev.export.find_default_export(nb['cells'])
    return default

def get_module_name():
    return find_default_export_for_notebook(get_notebook_path())

In [14]:
#hide
get_module_name()

'core'

In [15]:
#export

import os
import subprocess
import io

from threading import Thread
from queue import Queue 

def _reader(pipe, queue, pipe_name):
    try:
        with pipe:
            for line in iter(pipe.readline, b''):
                queue.put((pipe_name, line))
    finally:
        queue.put(None)
        
def run_process(args, **kwargs):
    proc = subprocess.Popen(
        args, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.PIPE, 
        bufsize=1)
    
    q = Queue()
    Thread(target=_reader, args=[proc.stdout, q, 'stdout']).start()
    Thread(target=_reader, args=[proc.stderr, q, 'stderr']).start()
    #for _ in range(2):
    for source, line in iter(q.get, None):
        line = line.decode('utf-8').rstrip('\r\n')
        if source == 'stderr':
            print("\x1b[31m{}\x1b[0m".format(line))
        else:
            print(line)
    
    return proc.poll()

In [17]:
#hide
run_process(['ls'])

00_core.ipynb
01_ai_platform_constants.ipynb
ai_platform_runner.ipynb
CONTRIBUTING.md
Dockerfile
docs
entry_point.ipynb
gcp_runner
index.ipynb
kubernetes_runner.ipynb
LICENSE
local_runner.ipynb
logfile.txt
MANIFEST.in
poll_kubernetes_logs.sh
README.md
sample_code.ipynb
settings.ini
setup.py
template.yaml.jinja


0

In [30]:
#export

def get_run_python_args(func, **kwargs):
    package_name = get_package_name()
    module_name = get_module_name()
    function_name = func.__name__
    args = ['python', 
         '-u',
         '-m', 
         'gcp_runner.entry_point',
         '--module-name=%s' % package_name + '.' + module_name,
         '--function-name=%s' % function_name]
    return args

In [33]:
export_and_reload_all()

Converted 00_core.ipynb.
Converted 01_ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code.ipynb.
